<a href="https://colab.research.google.com/github/taimoorsardar/Question-Answering-WebSrc/blob/main/QuestionAnswerWebSRC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install bs4
!pip install tqdm

in the file_url variable, insert the link to your WebSrc dataset folder downloaded in your drive

In [ ]:
import gdown

# Google Drive file URL
file_url = ""

# Output file path
output_path = "web.zip"  # Change the extension based on the actual file type

# Download the file
gdown.download(file_url, output_path, quiet=False)




In [ ]:
!unzip web.zip

Archive:  web.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of web.zip or
        web.zip.zip, and cannot find web.zip.ZIP, period.


In [ ]:
from bs4 import BeautifulSoup

def extract_core_dom_from_file(html_file_path, max_length=1024):

    # Read the HTML content from the file
    with open(html_file_path, "r", encoding="utf-8") as html_file:
        html_content = html_file.read()

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Extract the core DOM structure (e.g., body or main container)
    core_dom = soup.body  # You can adjust this to match your specific needs

    # Extract plain text content from the HTML
    core_dom = core_dom.get_text(separator='')

    # Convert the core DOM back to HTML
    core_dom_html = str(core_dom)

    # Truncate the HTML to fit within the specified max_length
    if len(core_dom_html) > max_length:
        core_dom_html = core_dom_html[:max_length]

    return core_dom_html




In [ ]:
import csv

def extract_questions_and_answers(csv_file_path, html_data):
    questions = []
    answers = []
    contexts = []
    with open(csv_file_path, "r", newline="", encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            question = row['question'].strip()
            answer = row['answer'].strip()
            id = row['id'].strip()[2:9]
            questions.append(question)
            answers.append(answer)
            contexts.append(html_data[id])

    return contexts, questions, answers

In [ ]:
# delete dataset.csv file from release folder before running this file

import json
import os
def extract_to_json(input_dir):
        contexts = []
        questions = []
        answers = []
        html_data = {}
        data = {}
        csv_paths = []
        count = 0
        for website_type in os.listdir(input_dir):
          count += 1
          current_type_path = os.path.join(input_dir, website_type)
          print(current_type_path)
          current_type = os.listdir(current_type_path)
          for website_num in current_type:
            current_num_path = os.path.join(current_type_path, website_num)
            current_num = os.listdir(current_num_path)
            csv = os.path.join(current_num_path, 'dataset.csv')
            csv_paths.append(csv)
            current_folder = os.path.join(current_num_path, 'processed_data')
            for filename in os.listdir(current_folder):
              current_file = os.path.join(current_folder, filename)
              if filename.lower().endswith(".html"):
                  context = extract_core_dom_from_file(current_file)

                  html_data[filename[:-5]] = context
          if count == 6:
            break

        for data_path in csv_paths:

          context, question, answer = extract_questions_and_answers(data_path, html_data)
          contexts += context
          questions += question
          answers += answer

        for id in range(len(questions)):
          entry = {
              'context': contexts[id],
              'question': questions[id],
              'answer': answers[id]
          }
          data[id] = entry

        with open('/content/release/data.json', "w") as json_file:
          json.dump(data, json_file, indent=4)

extract_to_json('/content/release/')


In [ ]:
# Load the dictionary from the JSON file
with open('/content/release/data.json', 'r') as json_file:
    loaded_dict = json.load(json_file)

# Now you can work with the loaded_dict
print(loaded_dict['5']['context'])

    Food for Camping Vol 2    The Australian Women's Weekly                                Write a review         Food for Camping Vol 2    The Australian Women's Weekly     PaperBack   August 4, 2020      The sequel to our best-selling Food for Camping, this book contains a whole new collection of outdoor meal ideas, perfect for both the well-seasoned camper and those hitting the trails for the first time. Using portable and accessible ingredients and practical cooking techniques, you don't have to worry about being strapped for ideas when it come to your next camping trip       


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm  # Import the tqdm library for progress bar
import json

json_file_path = '/content/release/data.json'

class QADataset(Dataset):
    def __init__(self):
        with open(json_file_path, 'r') as json_file:
            self.loaded_dict = json.load(json_file)

    def __len__(self):
        return len(self.loaded_dict)

    def __getitem__(self, idx):
        return self.loaded_dict[str(idx)]['context'], self.loaded_dict[str(idx)]['question'], self.loaded_dict[str(idx)]['answer']

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available, using GPU.")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")

# Load pre-trained model and tokenizer
model_name = 't5-small'
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
model_state_dict = torch.load("/content/drive/MyDrive/Colab Notebooks/Models/model_weights_epoch5.pth")
model.load_state_dict(model_state_dict)

tokenizer = T5Tokenizer.from_pretrained(model_name)

# Create dataset and dataloader
dataset = QADataset()
dataloader = DataLoader(dataset, batch_size=12, shuffle=True)

# Fine-tuning loop with progress bar and evaluation metrics
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(1):  # Adjust the number of epochs as needed
    total_loss = 0.0
    all_predictions = []
    all_labels = []

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{5}", unit="batch")

    for batch in progress_bar:
        batch_contexts, batch_questions, batch_answers = batch
        inputs = [f"context: {ctx} question: {ques}" for ctx, ques in zip(batch_contexts, batch_questions)]
        input_ids = tokenizer(inputs, max_length = 1024, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True).to(device)
        labels = tokenizer(batch_answers, padding=True, truncation=True, max_length=2048, return_tensors="pt")["input_ids"].to(device)

        optimizer.zero_grad()
        outputs = model(**input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Generate predictions and labels for evaluation metrics
        with torch.no_grad():
            generated_ids = model.generate(input_ids=input_ids.input_ids)
            predictions = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]
            all_predictions.extend(predictions)
            all_labels.extend(batch_answers)

    # Calculate evaluation metrics
    precision = precision_score(all_labels, all_predictions, average='macro')
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')
    exact_match = sum(1 for pred, true in zip(all_predictions, all_labels) if pred == true) / len(all_labels)

    avg_loss = total_loss / len(dataloader)
    progress_bar.set_postfix(loss=avg_loss, precision=precision, recall=recall, f1=f1, exact_match=exact_match)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Exact Match: {exact_match:.4f}")


GPU available, using GPU.


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Epoch 1/5:   0%|          | 0/15416 [00:00<?, ?batch/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Epoch 1/5:   0%|          | 1/15416 [00:10<43:11:53, 10.09s/batch]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1254: UserWarning: Using the mod

Epoch 1, Loss: 0.1050, Precision: 0.5192, Recall: 0.4976, F1: 0.4996, Exact Match: 0.7791


In [ ]:
def generate_answers(model, tokenizer, question, context):
    input_text = f"answer: {question} context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", add_special_tokens=True).to(device)

    with torch.no_grad():
        output_ids = model.generate(input_ids)

    generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_answer

# Example usage
question = 'What is the website? '
context = extract_core_dom_from_file('/content/release/university/03/processed_data/0300001.html')

generated_answer = generate_answers(model, tokenizer, question, context)

print("Generated Answer:", generated_answer)


Generated Answer: www.adler.edu


In [ ]:
# Save model weights
# torch.save(model.state_dict(),'/content/drive/MyDrive/Colab_Notebooks/Models/t5-small_model.pth')

In [ ]:
# Save model weights
model_state_dict = model.state_dict()
torch.save(model_state_dict, 'model_weights_epoch6.pth')


In [ ]:
from google.colab import files
files.download('model_weights_epoch6.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import runtime
runtime.unassign()